# Programowanie obiektowe w języku *C#*

## Wstęp

Klasa to pojemnik na dane i metody. Może posiadać:

* nazwę,
* pola,
* właściwości,
* zdarzenia,
* konstruktory/destruktory,
* metody (w tym przedefiniowane operatory np. dodawania).

*Nazwa* to identyfikator typu klasy. W momencie tworzenia instancji klasy nazwa ta jest przekazywana za słowem kluczowym `new`. Pole to zmienna, która przynależy do klasy i zawiera dane, które są procesowane przez funkcje. *Właściwości* podobnie jak pola zawierają dane, ale mogą zawierać dodatkowe fragmenty kodu. Mogą posiadać dodatkowe modyfikatory dostępu. *Zdarzenia* pozwalają użytkownikowi podłączyć się do powiadomień, które klasa wyzwala. Używane są głównie w programowaniu zdarzeniowym oraz programowania `GUI`. *Konstruktor* i *destruktor* zawierają instrukcje, które zostaną wykonane w momencie kolejno, tworzenia i zwalniania instancji klasy. Zwykle *konstruktor* zawiera fragmenty kodu odpowiedzialne za przydział zasobów i inicjalizację zmiennych, a *destruktor* zwalnianie zasobów niezarządzanych (które nie są zarządzane przez `garbage collector`). *Metody* to funkcje, które są własnością klasy.

Poniższy listing przedstawia przykładową deklarację i użycie klasy: 

In [331]:
public class SimpleClass //nazwa
{
   public int Field; //pole
   public readonly Field; //tylko do odczytu oprócz konstruktora
   public int Property { get; set; } //właściwość
   public int PropWithDefault { get; set; } = 12; //właściwość zainicjowana
   public int PropWithInit { get; init; } //tylko do odczytu oprócz wyrażenia tworzącego instancję obiektu
   public SimpleClass(){} //konstruktor
   ~SimpleClass(){} //destruktor
   public void DoSomething(int field) //funkcja klasy
   {
      Field = field;
   }
}

SimpleClass instance1 = new SimpleClass(); //nowa instancja
var instance2 = new SimpleClass(); //nowa instancja 2
SimpleClass instance1 = new(); //nowa instancja 3 (C# 9)

Error: (4,25): error CS1519: Nieprawidłowy token „;” w deklaracji składowej klasy, rekordu, struktury lub interfejsu
(4,25): error CS1519: Nieprawidłowy token „;” w deklaracji składowej klasy, rekordu, struktury lub interfejsu

Słowo kluczowe `init` oznacza tutaj, że tylko w wyrażeniu, które tworzy obiekt, można przypisać zmiennej wartość.

Należy oddzielić deklarację klasy od jej instancji. Klasa to byt logiczny, jego fizyczną reprezentację stanowi instancja, która znajduje się w pamięci komputera.

## Funkcje i ich argumenty

Argumenty do funkcji można przekazywać na wiele sposób. W języku C# domyślnie argumenty przekazywane są przez wartość. Wewnątrz funkcji każdy taki obiekt jest kopią zmiennej, która jest przekazywana do funkcji. W przypadku typów prostych (value type jak int, float, double) zmiana wartości takiej zmiennej wewnątrz funkcji nie powoduje zmiany przekazywanej funkcji. Jest to prawidłowe zachowanie, gdyż zgodnie z dobrymi praktykami, funkcja powinna przetwarzać dane, na podstawie parametrów przekazywanych jako argumenty. Ilustruje to poniższy przykład.

In [332]:
public void PassValueType(int argument) {
    argument = 2;
    Console.WriteLine($"PassValueType argument: {argument}");
}

var argument = 1;

PassValueType(argument);

Console.WriteLine($"Argument: {argument}");

PassValueType argument: 2
Argument: 1


Powyższy problem można rozwiązać na trzy sposoby. 

1. Funkcja może zwracać zmodyfikowany argument.
2. Jeśli funkcja zmienia argument oznacza to, że może zostać ona wydzielona jako pole lub właściwość całej klasy, a nie zmienna lokalna.
3. Można dodać słowo kluczowe **`ref`** w ten sposób mówiąc kompilatorowi, że ma przekazać adres zmiennej, a nie tworzyć jej kopii.

Użycie słowa kluczowego **`ref`** ma dwa zastosowania, jedno popularne (dla typów prostych), drugie mniej znane (dla typów referencyjnych jak klasa). Najpierw zostanie omówione pierwsze zastosowanie, które naprawia powyższy problem z przekazywaniem kopii zmiennej.

In [333]:
public void PassValueTypeByRef(ref int argument) {
    argument = 2;
    Console.WriteLine($"PassValueTypeRef argument: {argument}");
}

var argument = 1;

PassValueTypeByRef(ref argument);

Console.WriteLine($"Argument: {argument}");

PassValueTypeRef argument: 2
Argument: 2


Jak widać na powyższym przykładzie, słowo kluczowe **`ref`** należy użyć w dwóch miejscach, w momencie wywoływania funkcji i jej deklaracji. 

Drugim mniej istotnym, a co za tym idzie mniej popularnym zastosowaniem jest inicjalizacja klasy. Bez słowa kluczowego **`ref`**, kompilator przekaże kopię zmiennej wskaźnikowej na typ. Problem pojawia się w momencie inicjalizacji klasy i przekazania adresu do utworzonej kopii zmiennej wewnątrz funkcji. Ilustruje to poniższy kod.

In [334]:
public class ClassContainer 
{
    public int Value { get; set; }
}

public void PassRefType(ClassContainer container) 
{
    container.Value = 2;
    Console.WriteLine($"PassRefType argument: {container.Value}");

    container = new ClassContainer() { Value = 3 };
    Console.WriteLine($"PassRefType argument: {container.Value}");
}

var container = new ClassContainer() { Value = 1 };

Console.WriteLine($"Argument before call: {container.Value}");

PassRefType(container);

Console.WriteLine($"Argument after call: {container.Value}");

Argument before call: 1
PassRefType argument: 2
PassRefType argument: 3
Argument after call: 2


Jak widać na powyższym listingu, w momencie wywołania funkcji powstaje kopia wskaźnika (referencji) do obiektu utworzonego przed wywołaniem funkcji `PassRefType`. Wewnątrz tej funkcji, zmiana wartości zmiennej `value` powoduje zmianę globalnie, co łatwo zaobserwować po tym co zostało wydrukowane na ekranie po wywołaniu funkcji (ostatnia linia kodu). Wracając jednak do funkcji `PassRefType`, po drugiej inicjalizacji obiektu klasy `ClassContainer` wskażnik `container` zawiera już inny adres, niż początkowo. Jednak zmienia się kopia wskaźnika widoczna w przestrzeni nazw funkcji `PassRefType`, a nie wskaźnik `container` zainicjowany jako pierwszy. Problem ten jednak łatwo naprawić przy użyciu **`ref`**, który powoduje przekazanie wskaźnika na wskaźnik, co przedstawia poniższy kod.

In [335]:
public class ClassContainer 
{
    public int Value { get; set; }
}

public void PassRefType(ref ClassContainer container) 
{
    container.Value = 2;
    Console.WriteLine($"PassRefType argument: {container.Value}");

    container = new ClassContainer() { Value = 3 };
    Console.WriteLine($"PassRefType argument: {container.Value}");
}

var container = new ClassContainer() { Value = 1 };

Console.WriteLine($"Argument before call: {container.Value}");

PassRefType(ref container);

Console.WriteLine($"Argument after call: {container.Value}");

Argument before call: 1
PassRefType argument: 2
PassRefType argument: 3
Argument after call: 3


Zmienna container wskazuje na inny obszar pamięci, co łatwo zaobserwować po ostatnim wydruku na ekran wartości zmiennej `value`.

Kolejnym słowem kluczowym związanym z przekazywaniem argumentów do funkcji jest **`out`**, który wymusz przekazanie wprost wartości dla zmiennej z tym słowem kluczowym w ciele wywoływanej funkcji.

In [336]:
public void PassWithOut(out string name) {
    name = "Ala";
}

string name = null;

PassWithOut(out name);

Słowo kluczowe **`out`** musi pojawić się w dwóch miejscach, w deklaracji funkcji i jej wywołaniu. Nieprzekazanie tego słowa kluczowego w jednym z miejsc powoduje błąd kompilacji.

Dobrą praktyką w przypadku bardziej złożonych funkcji jest w deklaracji funkcji ustawienie parametrów domyślnych. Poniżej znajduje się przykładowa deklaracja tego typu parametrów.

In [337]:
public void PassWithDefaults(string label, int n = 1, int length = 2) {
    for (int i = 0; i < n; i++) {
        Console.WriteLine(label.Substring(0, length));
    }    
}

PassWithDefaults("Hello World", length: 5);

Hello


Do funkcji `PassWithDefaults` został przekazany łańcuch znaków "Hello World" wraz z drugim domyślnym parametrem `length`. Minimalnie do funkcji należy przekazać parametr `label`. Pozostałe parametry są opcjonalne z wartościami domyślnymi.

Ostatnim sposobem na przekazywanie parametrów jest użycie słowa kluczowego **`params`**, które umożliwia agregowanie argumentów w tablice, dzięki czemu w trakcie wywołania funkcji można przekazywać dowolnie wiele argumentów po przecinku.

In [338]:
public void PassWithParams(string label, params int[] indexes) {
    foreach(var index in indexes) {
        Console.WriteLine(label[index]);
    }
}

PassWithParams("Hello World", 0, 1, 2, 3, 4);

H
e
l
l
o


## Przeciążenie funkcji

Jedną z cech języka C# jest możliwość przeciążenia funkcji (oraz konstruktorów). Oznacza to, że wiele zdefiniowanych funkcji ma takie same nazwy, ale różne argumenty lub co najmniej różne typu argumentów. Wywołanie funkcji musi być jednoznaczne, dlatego w przypadku niejednoznaczności kompilator zwróci błąd.

In [339]:
public void Log(string message) {    
    Console.WriteLine(message);
}

public void Log(Exception exception) {
    Console.WriteLine(exception.Message);
}

public void Log(Exception exception, string customPrefix) {
    Console.WriteLine($"{customPrefix}: {exception.Message}");
}

Log("Message");
Log(new Exception("Message from exception"));
Log(new Exception("Message from exception"), "Prefix");

Message
Message from exception
Prefix: Message from exception


## Enkapsulacja (hermetyzacja)

Oznacza ukrywanie funkcji lub zmiennych klasy, aby były dostępne tylko dla obiektów dziedziczących oraz ogranicza dostępu z zewnątrz. Dostępne są następujące modyfikatory dostępu:

* `public`.
* `private`.
* `protected`.
* `internal`.
* `protected internal`.
* `private protected (C# 7.2)`

Poniższa tabela przedstawia modyfikatory oraz ich ograniczenia w dostępie do ich zasobów.

|Modyfikator|Klasy pochodne z tego samego modułu|Pozostałe klas z tego samego modułu (projektu)|Klasy pochodne z innych modułów|Klasy z innych modułów|
|:-|:-:|:-:|:-:|:-:|
|`private`              |Nie|Nie|Nie|Nie|
|`public`               |Tak|Tak|Tak|Tak|
|`protected`            |Tak|Nie|Tak|Nie|
|`internal`             |Tak|Tak|Nie|Nie|
|`protected internal`   |Tak|Tak|Tak|Nie|
|`private protected`    |Tak|Nie|Nie|Nie|

Modyfikator `protected internal` zawiera sumę logiczną `protected` i `internal`. Z kolei `private protected` umożliwia dostęp tylko klasą pochodnym w ramach tego samego modułu.

Przykładowe użycie dowolnego modyfikator może wygląda następująco:

In [340]:
public class SimpleClass
{
   private int Field; 
   public int Property { get; protected set; }
   public SimpleClass(){} //konstruktor
   ~SimpleClass(){} //destruktor
   public void DoSomething(int field)
   {
      Field = field;
   }
}

## Dziedziczenie

Język C# umożliwia dziedziczenie jednokrotne (dla klas) i wielokrotne dla interfejsów. Dostępne są również konstrukcje języka `base` (odwołanie do instancji bazowej) i `this` (odwołanie do bieżącej instancji). Dziedziczenie powoduje tworzenie klasy szczegółowej na podstawie ogólnej klasy bazowej. Poniżej znajduje się przykład dziedziczenia:

In [341]:
public class SpecificClass: SimpleClass
{
   private int Field2; 
   
   public SpecificClass()
   : base() 
   {
      base.Property = 2;
      this.Field2 = 2;
   }
}

Dziedziczymy pola, metody, właściwości i zdarzenia. W zależności od modyfikatora dostępu do tych elementów, w klasie pochodnej możemy z nich skorzystać lub nie.

## Polimorfizm

Pozwala przeciążać metody bazowe umożliwiając ich uszczegółowienie lub zmianę znaczenia. Przykładowo:

In [342]:
public class A
{
   public virtual void Method1() { }
}
public class B : A
{
   public override void Method1() 
   { 
      base.Method1();//metoda bazowa
   }
}		

## Abstrakcja

Poprzez abstrakcję należy rozumieć pewien interfejs, który musi być zaimplementowany w klasie potomnej. Gwarantuje to występowanie metod i właściwości w klasach potomnych. Interfejs to szablon i nie może posiadać instancji. Podobnie jak klasa abstrakcyjna, która może oprócz interfejsu posiadać implementację niektórych metod. Może również wymuszać na obiektach dziedziczących implementacje metod bądź pól.

In [343]:
public interface SimpleInterface
{
   Int32 Property { get; set; }
   void Method1();        
}

public abstract class SimpleAbstract
{
   Int32 Property { get; set; }
   public abstract void Method1();

   public void Method2()
   {
   }
}

## Typy dynamiczne

Umożliwiają w czasie wykonania aplikacji tworzenie klasy. Należy odróżnić słowo kluczowe `var` od deklaracji zmiennej dynamicznej `dynamic`. W pierwszym przypadku kompilator przekaże za nas typ. Jest to skrócony zapis konkretnego typu znanego w czasie kompilacji. W drugim przypadku typ tworzony jest w czasie działania programu. Zbyt częste użycie typu dynamicznego może spowolnić działanie programu. Dodatkowo typ `dynamic` może być użyty jako typ parametru funkcji.

In [344]:
dynamic dynClass = new
{
   Prop1 = 12,
   Prop2 = "Test"
};

## Static

Wszystkie elementy klasy mogą być statyczne. Statyczność klasy powoduje, że istnieje tylko jedna instancja. Tworzeniem klas statycznych zajmuje się środowisko wykonawcze platformy `.NET`. Innym sposobem na tworzenie tylko jednej instancji klasy jest wzorzec projektowy `singleton`:

In [345]:
public class JustOne
{
    private static Single singleInstance;
    private JustOne() {} //konstruktor prywatny    
    public static JustOne GetInstance() 
    {
      singleInstance = singleInstance ?? new JustOne();
      return singleInstance;
    }
}

Error: (7,24): error CS0019: Nie można zastosować operatora „??” do argumentów operacji typu „float” lub „JustOne”.
(8,14): error CS0029: Nie można niejawnie przekonwertować typu „float” na „JustOne”.

Operator `??` jest operatorem binarnym. W przypadku, gdy jego lewy argument ma wartość `null` wykonuje instrukcje zawarte w prawym argumencie.

Klasy statyczne mogę też zastępować typ wyliczeniowy `enum` np.:

In [346]:
public static class PersonType
{
    public static readonly string Teacher = "T";
    public static readonly string Student = "S";
}

## Zdarzenia

Zdarzenia deklarujemy podobnie jak pola klasy. Służą do wywoływania metod do obsługi tych zdarzeń z klas zewnętrznych. Przykładowo klasa realizująca zapis do pliku może wywołać zdarzenie `Complete`. Poniższy listing przedstawia obsługę zdarzenia:

In [347]:
public class Writer
{
  public EventHandler<bool> Complete;

  public void Write(string fileName)
  {
    OnComplete(true);
  }

  public void OnComplete(bool status)
  {
    if (Complete != null)
      Complete(this, status);
    //lub prosciej
    Complete?.Invoke(this, status);
  }
}

static void Main(string[] args)
{
    var writer = new Writer();
    writer.Complete += (source, status) =>
        Console.WriteLine("Write: {0}", status);
    
    writer.Complete += AnotherHandler;

    writer.Write("file.txt");
}
static void AnotherHandler(object source, bool status)
{
	Console.WriteLine("Write: {0}", status);
}

Wewnętrznie dodanie metody obsługi do zdarzenia realizowane jest podobnie do właściwości z tą różnicą, że zdarzenie posiada metody `add()` i `remove()`. Zamiast użycia klasy `EventHandler` można stworzyć własny szablon tzw. deletegat. Tworzenie delegata ilustruje poniższy listing:

In [348]:
delegate void SomeHandler(object sender, bool status);

Delegaty to szablony metod. Przypisanie metody do obsługi zdarzenia musi posiadać taką samą sygnaturę co delegat tzn. posiadać ten sam zwracany typ oraz te same argumenty.

## Typy generyczne

Deklaracja klasy generycznej wygląda bardzo podobnie do zwykłej klasy, ale posiada
dodatkowe argumenty. Klasa generyczna jest odpowiednikiem szablonu w c++.

In [349]:
public class SimpleClass<T>
{
   public T Field; 
   public T Property { get; set; }
   
   public void DoSomething(T field)
   {
      Field = field;
   }
}

## Ograniczenia argumentu generycznego

Projektując klasy możemy wymusić typ argumentu dla klasy generycznej. Można zdefiniować następujące ograniczenia:

* `T`: `struct` - argument musi być typem prostym,
* `T` : `class` - argumentem musi być klasa,
* `T` : `new()` - argument musi posiadać konstruktor bezparametrowy,
* `T` : `base_class_name` - argument musi dziedziczyć po *base_class_name*,
* `T` : `interface_name` - argument musi implementować *interface_name*,
* `T` : `U` - argument `T` musi dziedziczyć po argumencie `U`.

In [350]:
public class GenericClass<T> where T : Student

Error: (1,47): error CS1514: Oczekiwano znaku {
(1,47): error CS1513: Oczekiwano znaku }

## Indeksery

Każda klasa może posiadać tzw. indekser. Użycie indeksera przypomina użycie tablicy. Przykładowy indekser przedstawia poniższy listing:

In [351]:
public T this[int i]
{
   get
   {            
   }
   set
   {            
   }
}
...
IntCollection<int> col = new IntCollection<int>();
col[0] = 12;

Error: (10,1): error CS8635: Nieoczekiwana sekwencja znaków „...”
(10,3): error CS1002: Oczekiwano średnika (;)
(10,3): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku

## *Tuple*

W przypadku, gdy funkcje zawierają więcej, niż jedną zmienną można utworzyć dedykowaną klasę lub krotkę (ang. *tuple*) w wersji nazwanej (z podanymi nazwami pól w deklaracji) lub nienazwanej (kolejne pola nazywają się *`item1`*, *`item2`* itd).

In [352]:
public (string name, string surname) tuple_function() {
    return ("Jan", "Kowalski");
}

var person = tuple_function(); 

Console.WriteLine($"Person 1: {person.name} {person.surname}");

(string name, string surname) = tuple_function(); //unboxing

Console.WriteLine($"Person 2: {name} {surname}");

Person 1: Jan Kowalski
Person 2: Jan Kowalski


## Rekord

Rekordy postały z myślą o nowych mechanizmach języka. Pod względem kopiowania (przypisywania) zachowują się jak struktury, ale są typami referencyjnymi. Poniżej znajdują się przykładowe operacje na tych typach niemożliwe do użycia z klasami czy strukturami.

In [353]:
public record StudentRecord 
{ 
    public string FirstName { get; init; } 
    public string LastName { get; init; }
    public int Id { get; init; }
    public StudentRecord(string firstName, string lastName, int id) 
      => (FirstName, LastName, Id) = (firstName, lastName, id);
    public void Deconstruct(out string firstName, out string lastName, out int id) // jak ma się zachowywać klasa w przypadku dekonstrukcji
      => (firstName, lastName, id) = (FirstName, LastName, Id);
}

public record StudentRecord2(string FirstName, string LastName, int No); //tożsame z powyższym

var student = new StudentRecord("Mads", "Torgersen", 1);
var (f, l, id) = student; 

var student2 = student with { LastName="Kowalski", Id=2 };

Console.WriteLine(student);
Console.WriteLine(student2);

StudentRecord { FirstName = Mads, LastName = Torgersen, Id = 1 }
StudentRecord { FirstName = Mads, LastName = Kowalski, Id = 2 }


## Przydatne techniki wbudowane w język

Operując na plikach należy bezwzględnie zwalniać systemowy uchwyt do pliku metodą `Close()`. Nie wykonanie tej czynności powoduje, że pliku nie da się skasować oraz nadpisać. W przypadku, gdy klasa posiada zaimplementowaną metodę `Dispose()` można użyć konstrukcji:

In [354]:
using (StreamWriter outFile = new StreamWriter(path))
{
   outFile.WriteLine("lancuch");
}

Error: (1,48): error CS0103: Nazwa „path” nie istnieje w bieżącym kontekście

W powyższym listingu nie ma konieczności wywoływania metody `Close()` ponieważ zajmie się tym metoda `Dispose` wywołana niejawnie przy zakończeniu bloku "\}". Powyższy zapis jest równoważny:

In [355]:
try
{
  StreamWriter outFile = new StreamWriter(path);
  outFile.WriteLine("string data");  
}
finally
{
  if(outFile != null)
    outFile.Dispose();
}

Error: (3,43): error CS0103: Nazwa „path” nie istnieje w bieżącym kontekście
(8,6): error CS0103: Nazwa „outFile” nie istnieje w bieżącym kontekście
(9,5): error CS0103: Nazwa „outFile” nie istnieje w bieżącym kontekście

## Interpolacja łańcuchów znaków c.d.

Jedną z nowych funkcji w specyfikacji języka jest technika interpolacji łańcuchów znaków, bardzo często spotykana we nowoczesnych językach programowania, takich jak *Python*. Zamiast używać operatora plus do łączenia napisów, programista może sformatować napis. W poprzednim pliku temat został już omówiony, w tym miejscu zostały jedynie dodane inne operacje formatowania. 

In [356]:
int value = 1;
double value2 = 2.12345;

Console.WriteLine($"Integer {value}, double: {value2} and double formated one: {value2:0.0}");
var date = DateTime.Now;
Console.WriteLine($"Now is: {date:HH:mm}")

Integer 1, double: 2,12345 and double formated one: 2,1
Now is: 00:59


## Przydatne klasy

Najczęściej wykorzystywanymi klasami są lista (`List`, `Queue`, `Array`). Przykład użycia listy:

In [357]:
//lista
List<Int32> list = new List<int>();
list.Add(12);
//kolejka
Queue<Int32> queue = new Queue<int>();
queue.Enqueue(12);
Int32 last = queue.Dequeue();

### Klasa `ConfigurationManager`

Często spotykanym podejściem do tworzenia oprogramowania jest uzależnienie wykonania programu od globalnych parametrów wejściowych. Jest to bardzo wygodne podeście ze względu na cykl życia aplikacji. Przykładowo, gdy program łączy się z bazą danych, na etapie testowania połączenie powinno odbywać się z instancją deweloperską bazą danych. Z kolei, gdy program zostanie wdrożony na środowisko produkcyjne, program powinien łączyć się z instancją produkcyjną bazy danych. W środowisku `.NET` powstała specjalna klasa `ConfigurationManager`, która umożliwia w łatwy sposób korzystanie z pliku konfiguracyjnego w formacie `XML` `App.config`. Po etapie kompilacji projektu konfiguracja zostanie przeniesiona do pliku `plik.exe.config`. Klasa `ConfigurationManager` dostępna jest po dodaniu referencji `System.Configuration` do projektu. Można to zrobić klikając prawym przyciskiem myszy na projekt i wybranie opcji `Add` `Reference...`. Poniższy listing przedstawia użycie klasy:

In [358]:
public static void Main(string[] args)
{  
  var strCount = ConfigurationManager.AppSettings["Count"];
  var count = int.Parse(strCount);
}

Error: (3,18): error CS0103: Nazwa „ConfigurationManager” nie istnieje w bieżącym kontekście

Przy takiej konfiguracji plik `App.config` powinien wyglądać następująco:

In [359]:
<?xml version="1.0" encoding="utf-8" ?>
<configuration>
  <appSettings>
    <add key="Count" value="20"/>
  </appSettings>
</configuration>

Error: Unexpected token '<'

### Klasa `StringBuilder`
W przypadku konkatenacji wielu łańcuchów znaków należy używać klasy `StringBuilder`:

In [360]:
StringBuilder builder = new StringBuilder();
   
foreach(var person in collection)
{
   builder.Append(person.Name);
}      

Error: (3,23): error CS0103: Nazwa „collection” nie istnieje w bieżącym kontekście

### Funkcja `Format`
Formatowanie tekstu to jedno z najczęstszych zadań programistycznych. Z tego powodu na platformie `.NET` udostępniono metodę `format`. Jest ona zaimplementowana zarówno w klasie `StringBuilder` w metodzie `AppendFormat()` jak i `string.Format()`:

In [361]:
//wyswietlenie
string.Format("First argument {0}, drugi {1}"), 1,2);
//zaokraglenie
string.Format("First argument {0:0.00}"), 1.255555);
//daty
string.Format("{0:yyyy-MM-dd}"), DateTime.Now());
//dopelnienie ciagu znakow
string.Format("{0,22}", "Znaki");
string.Format("{0,22}", "Znaki_Znaki");

Error: (2,47): error CS1002: Oczekiwano średnika (;)
(2,47): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(2,50): error CS1002: Oczekiwano średnika (;)
(2,50): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(2,52): error CS1002: Oczekiwano średnika (;)
(2,52): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(4,41): error CS1002: Oczekiwano średnika (;)
(4,41): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(4,51): error CS1002: Oczekiwano średnika (;)
(4,51): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(6,32): error CS1002: Oczekiwano średnika (;)
(6,32): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku
(6,48): error CS1002: Oczekiwano średnika (;)
(6,48): error CS7017: Oczekiwano definicji składowej, instrukcji albo znacznika końca pliku

## Zadania do wykonania

### Zadanie 1

Zaprojektuj klasę abstrakcyjną / rekord, która wyznacza pole, obwód oraz zwraca długości boków trójkąta (dowolnego). Klasami dziedziczącymi z tej klasy ma być trójkąt:
* równoboczny (konstruktor z jednym parametrem),
* równoramienny (konstruktor z dwoma parametrami - dwa różne boki),
* prostokątny (konstruktor z z dwoma parametrami - długości przyprostokątne).

> Przeciąż metodę `ToString()`, która ma zawierać zestaw podstawowych informacji. Zadanie można zrealizować na dwa sposoby: 
1. metody do obliczeń pola i obwodu zaimplementować w klasie abstrakcyjnej używając wzoru:

In [362]:
public abstract class Trojkat{
public double a, b, c, obwod, pole;

public double ObliczPole(double a, double b, double c){
    double p = ObliczObwod(a, b , c) / 2;

        return Math.Round(Math.Sqrt(p*(p - a)*(p - b)*(p -c)), 2);

    }
public double ObliczObwod(double a, double b, double c){
    return a + b + c;
    }
public override String ToString(){
    return ($"Pole: " + pole + "\nObwod: " + obwod);
    }
}
public class TrojkatRownoboczny : Trojkat{

    public TrojkatRownoboczny(double a){
        this.a = a;
        this.obwod = ObliczObwod(a, a, a);
        this.pole = ObliczPole(a, a, a);
}
}

public class TrojkatRownoramienny : Trojkat{

    public TrojkatRownoramienny(double a, double b){
        this.a =a;
        this.b = b;
        this.obwod = ObliczObwod(a, b ,b);
        this.pole = ObliczPole(a, b, b);
    }
}

public class TrojkatProstokatny : Trojkat{
    public TrojkatProstokatny(double a, double b){
        this.a = a;
        this.b = b;
        this.c = Math.Sqrt(a*a + b*b);

        this.obwod = ObliczObwod(a, b, c);
        this.pole = ObliczPole(a, b, c);

    }



}


TrojkatRownoboczny trojkatRownoboczny = new TrojkatRownoboczny(5.0);
Console.WriteLine(trojkatRownoboczny);

TrojkatRownoramienny trojkatRownoramienny = new TrojkatRownoramienny(2.0,6.0);
Console.WriteLine(trojkatRownoramienny);

TrojkatProstokatny trojkatProstokatny = new TrojkatProstokatny(6.0,8.0);
Console.WriteLine(trojkatProstokatny);


Pole: 10,83
Obwod: 15
Pole: 5,92
Obwod: 14
Pole: 24
Obwod: 24


$
	P =\sqrt{p\cdot(p-A)\cdot(p-B)\cdot(p-C)}
$

gdzie: $p$ to połowa obwodu trójkąta. Symbole $A$, $B$, $C$ to długości poszczególnych boków. Oznaczyć metody obliczające pole i obwód jako abstrakcyjne i przeciążyć je w klasie szczegółowej.

Metoda `ToString` powinna być przeciążona w klasie abstrakcyjnej.

### Zadanie 2

Utwórz klasę generyczną kartoteka pracowników, która umożliwia:

* dodawanie/usuwanie,
* wyświetlanie,
* walidację istniejących już pracowników,
* wyszukiwanie danych. 

Klasa pracownik musi zawierać przynajmniej 5 cech. Każdy pracownik powinien posiadać stanowisko pracy. Klasa powinna zawierać walidację danych (poprzez metodę `Validate()`). Klasa pracownik powinna zawierać metody `Show()` i `IsMatch()`, z których będzie korzystała kartoteka przy wyszukiwaniu.

Przy wyświetlaniu pomocna może okazać się metoda `Format()`.

### Zadanie 3

Dodaj do kartoteki z pierwszego zadania odczytywanie i zapisywanie danych w formacie TXT, XML oraz JSON (minimum jeden). Wykorzystaj do tego celu wzorzec projektowy budowniczy. Abstrakcję konkretnego zapisu można przekazać do konstruktora kartoteki, która posiada metodę `Save()`.

### Zadanie 4
 
Rozszerz poprzednie zadanie o możliwość szyfrowania danych metodą cezara. Blok przesunięć powinien być większy, niż jeden. W celu jego przechowywania wykorzystaj klasę `ConfigurationManager` (najpierw należy dodać referencję do biblioteki `System.Configuration`).

In [2]:
// klasa pracownik z 5 cechami w tym stanowisko 
using System.Collections.Generic;
using System.IO;
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Xml.Serialization;
using System.Xml;


public class Pracownik {

    public String imie {get; set;}
    public String nazwisko {get; set;}
    public int wiek {get; set;}
    public String stanowisko {get; set;}
    public String plec {get; set;}

    // walidacja czy wszystkie dane są wprowadzone
    public bool Validate(){
        if (string.IsNullOrEmpty(imie) || string.IsNullOrEmpty(nazwisko) || wiek == 0 || string.IsNullOrEmpty(stanowisko)) {
            Console.WriteLine("Pracownik zawiera puste atrybuty");
            Console.WriteLine(Show());
            return false;
        }
        return true;
    }

    // metoda show wyświetlająca pracownika
    public string Show() {
        return string.Format(
            "| {0,-15}| {1,-15}| {2,-4}| {3,-15}| {4,-2}|",
            imie, nazwisko, wiek, stanowisko, plec);
    }

    // sprawdzenie czy istnieje juz pracownik
    public bool IsMatch(Pracownik pracownik){
        if( imie == pracownik.imie && 
            nazwisko == pracownik.nazwisko &&
            wiek == pracownik.wiek &&
            stanowisko == pracownik.stanowisko &&
            plec == pracownik.plec)
                return true;
            else return false;
    }

}

//Klasa generyczna kartoteka, pracownicy składowani w liście
public class Kartoteka<T> where T : Pracownik{

    List<T> baza {get; set;} = new List<T>();

    public void Dodaj(T pracownik) {
        if(pracownik.Validate() == true && IsExist(pracownik)) baza.Add(pracownik);
    }

    public void Usun(T pracownik) {
        baza.Remove(pracownik);
    }

    // czy juz jest obiekt o takich samych atrybutach
    public bool IsExist(T pracownik) {
        if (baza.Any(x => x.IsMatch(pracownik) == true)) {
            Console.WriteLine("\n Istnieje juz taki pracownik");
            Console.WriteLine(pracownik.Show());
            return false;
        }
        else return true;
    }

    // do wyswietlania
    public void Wyswietl() {
        Console.WriteLine(
            string.Format(
                " {0,-15}  {1,-15}  {2,-4}  {3,-15}  {4,-2}", 
                "imie", "nazwisko", "wiek", "stanowisko", "plec"));

        foreach(T a in baza) {
            Console.WriteLine(a.Show());
        }
    }

    public void Szukaj(String name) {
        Console.WriteLine("\nPracownik o imieniu:  \"{0}\":", name.ToLower());
        foreach(T a in baza) {
            if (a.Show().ToLower().Contains(name.ToLower()))
            {
                Console.WriteLine(a.Show());
            }
        }
    }    

    public bool Save(IBuilder builder) {
        return builder.Save(this);
    }

    public bool Read(IBuilder builder) {
        var loaded = builder.Read("Kartoteka.json");
        return true;
    }

    public interface IBuilder {
        bool Save(Kartoteka<T> baza);
        Kartoteka<T> Read(String fileName);
    }

    public class JSONBuilder : IBuilder {

        public bool Save(Kartoteka<T> kartoteka) {
            try {
                    var json = JsonSerializer.Serialize(kartoteka.baza);
                    File.WriteAllText("Kartoteka.json", json);
                }
                catch (Exception e) {
                    Console.WriteLine("\nZapisywanie się nie powiodło:");
                }
                Console.WriteLine("\nZapisano do pliku \"Kartoteka.json\"");
                return true;
        }

        public Kartoteka<T> Read(String fileName){
            String jsonString = "";
            try {
                jsonString = File.ReadAllText(fileName);
                Console.WriteLine("\nWczytano plik {0}", fileName);
            }
            catch (Exception e) {
                Console.WriteLine("\nWczytywanie się nie powiodło: {0}", e.Message);
            }
            var kartoteka = new Kartoteka<T>();
            kartoteka.baza = JsonSerializer.Deserialize<List<T>>(jsonString);
            return kartoteka;
        }

    }  

    public class TXTBuilder : IBuilder {

        public bool Save(Kartoteka<T> kartoteka) {
            try {
                using (StreamWriter sw = new StreamWriter("Kartoteka.txt")) {
                    foreach (var pracownik in kartoteka.baza) {
                        sw.WriteLine(pracownik.imie + "," + pracownik.nazwisko + "," + pracownik.wiek + "," + pracownik.stanowisko + "," + pracownik.plec);
                    }
                }
            }
            catch (Exception e) {
                Console.WriteLine("\nZapisywanie się nie powiodło: {0}", e.Message);
                return false;
            }
            Console.WriteLine("\nZapisano do pliku \"Kartoteka.txt\"");
            return true;
        } 

        public Kartoteka<T> Read(String fileName){
            var baza = new List<T>();
            try {
                using (StreamReader sr = new StreamReader(fileName)) {
                    string line;
                    while ((line = sr.ReadLine()) != null) {
                        var values = line.Split(',');
                        if (values.Length != 5) continue;
                        Pracownik pracownik = new Pracownik() {
                            imie = values[0],
                            nazwisko = values[1],
                            wiek = Int32.Parse(values[2]),
                            stanowisko = values[3],
                            plec = values[4]
                        };
                        baza.Add((T)pracownik);
                    }
                }
            }
            catch (Exception e) {
                Console.WriteLine("\nWczytywanie się nie powiodło: {0}", e.Message);
            }
            Console.WriteLine("\nWczytano plik \"{0}\"", fileName);
            var kartoteka = new Kartoteka<T>();
            kartoteka.baza = baza;
            return kartoteka;
        }

    }

    public class XMLBuilder : IBuilder {

        //TODO: 
        public bool Save(Kartoteka<T> baza) {
            try {
                XmlSerializer serializer = new XmlSerializer(typeof(Kartoteka<T>));
                using (StreamWriter sw = new StreamWriter("Kartoteka.xml")) {
                    serializer.Serialize(sw, baza);
                }
            }
            catch (Exception e) {
                Console.WriteLine("\nZapisywanie się nie powiodło: {0}", e.Message);
                return false;
            }
            Console.WriteLine("\nZapisano do pliku \"Kartoteka.xml\"");
            return true;
        }


        public Kartoteka<T> Read(String fileName) {
            return new Kartoteka<T>();
        }

    }

}

//stworzenie kartoteki oraz sprawdzanie walidacji
var kartoteka = new Kartoteka<Pracownik>();
kartoteka.Dodaj(new Pracownik(){imie = "Szymon", nazwisko = "Zielinski", wiek = 21, stanowisko = "Employee", plec = "m"});
kartoteka.Dodaj(new Pracownik(){imie = "Patryk", nazwisko = "Pirog", wiek = 23, stanowisko = "Manager", plec = "m"});
kartoteka.Dodaj(new Pracownik(){imie = "Patryk", nazwisko = "Pirog", wiek = 23, stanowisko = "Manager", plec = "m"});
kartoteka.Dodaj(new Pracownik(){imie = "Jan", nazwisko = "Kowalski", wiek = 28, stanowisko = "Employee", plec = "m"});
kartoteka.Dodaj(new Pracownik(){imie = "Krzysztof", nazwisko ="Nowak"});

Console.WriteLine("\n KARTOTEKA:");
kartoteka.Wyswietl();

// wyszukiwanie
kartoteka.Szukaj("Szymon");

// operacje na json
var jsonbuilder = new Kartoteka<Pracownik>.JSONBuilder();
kartoteka.Save(jsonbuilder);
Kartoteka<Pracownik> jsonpracownicy = jsonbuilder.Read("Kartoteka.json");
jsonpracownicy.Wyswietl();

// operacje na txt
var txtbuilder = new Kartoteka<Pracownik>.TXTBuilder();
kartoteka.Save(txtbuilder);
Kartoteka<Pracownik> txtpracownicy = txtbuilder.Read("Kartoteka.txt");
txtpracownicy.Wyswietl();

// operacje na xml
var xmlbuilder = new Kartoteka<Pracownik>.XMLBuilder();
kartoteka.Save(xmlbuilder);




 Istnieje juz taki pracownik
| Patryk         | Pirog          | 23  | Manager        | m |
Pracownik zawiera puste atrybuty
| Krzysztof      | Nowak          | 0   |                |   |

 KARTOTEKA:
 imie             nazwisko         wiek  stanowisko       plec
| Szymon         | Zielinski      | 21  | Employee       | m |
| Patryk         | Pirog          | 23  | Manager        | m |
| Jan            | Kowalski       | 28  | Employee       | m |

Pracownik o imieniu:  "szymon":
| Szymon         | Zielinski      | 21  | Employee       | m |

Zapisano do pliku "Kartoteka.json"

Wczytano plik Kartoteka.json
 imie             nazwisko         wiek  stanowisko       plec
| Szymon         | Zielinski      | 21  | Employee       | m |
| Patryk         | Pirog          | 23  | Manager        | m |
| Jan            | Kowalski       | 28  | Employee       | m |

Zapisano do pliku "Kartoteka.txt"

Wczytano plik "Kartoteka.txt"
 imie             nazwisko         wiek  stanowisko       plec
| S